In [1]:
import os
import pandas as pd;pd.__version__

'0.24.2'

In [66]:
data_path = r'E:\Dropbox\Trade\MarketData\700base\EURJPY.csv'
trade_path = r'E:\Dropbox\Trade\MarketData\700base\EURJPY_2.txt'

In [67]:
trade_df=pd.read_csv(trade_path, parse_dates=['EntryDateTime', 'ExitDateTime'])
trade_df['Direction'] = trade_df['Direction'].str.replace('TR Entry ', '')

In [68]:
trade_df.head()

,Ticker,EntryDateTime,Direction,EntryReason,EntryPrice,InitialStopLoss,InitialProfitTarget,TimeFrame,ExitDateTime,PreventStopLoss,...,ExitPrice,ExitReason,BgEntryBarState,BgEntryBarStatePrior,FtEntryBarState,FtEntryBarStatePrior,BgExitBarState,BgExitBarStatePrior,FtExitBarState,FtExitBarStatePrior
0,EURJPY,2018-06-18 00:02:18,Short,BG_BarFirstTrade,128.005,128.053786,127.670108,25,2018-06-18 00:07:03,128.053786,...,128.018000,BG40%Fade,BG_EntryBarState-Congestion,BG_EntryPriorBarState-Congestion,FT_EntryBarState-Congestion,FT_EntryPriorBarState-Congestion,BG_ExitBarStateCongestion,BG_ExitPriorBarStateCongestion,FT_ExitBarStateCongestion,FT_ExitPriorBarStateCongestion
1,EURJPY,2018-06-18 00:26:21,Short,BG_BarReEntryTrade,128.013,128.053786,127.670108,25,2018-06-18 00:34:52,128.093471,...,128.022000,BG40%Fade,BG_EntryBarState-Congestion,BG_EntryPriorBarState-Congestion,FT_EntryBarState-Congestion,FT_EntryPriorBarState-Congestion,BG_ExitBarStateCongestion,BG_ExitPriorBarStateCongestion,FT_ExitBarStateCongestion,FT_ExitPriorBarStateCongestion
2,EURJPY,2018-06-19 12:42:42,Long,BG_BarFirstTrade,127.157,127.096510,127.804960,25,2018-06-19 12:45:39,127.096510,...,127.148000,BG40%Fade,BG_EntryBarState-Transistion,BG_EntryPriorBarState-BreakOut,FT_EntryBarState-Congestion,FT_EntryPriorBarState-Congestion,BG_ExitBarStateTransistion,BG_ExitPriorBarStateBreakOut,FT_ExitBarStateCongestion,FT_ExitPriorBarStateCongestion
3,EURJPY,2018-06-21 05:56:31,Short,BG_BarFirstTrade,127.172,127.354119,126.749015,25,2018-06-21 06:54:30,127.354119,...,127.354119,StopLossHit,BG_EntryBarState-Congestion,BG_EntryPriorBarState-Congestion,FT_EntryBarState-Congestion,FT_EntryPriorBarState-Congestion,BG_ExitBarStateBreakOut,BG_ExitPriorBarStateCongestion,FT_ExitBarStateCongestion,FT_ExitPriorBarStateCongestion
4,EURJPY,2018-06-22 02:10:08,Long,BG_BarFirstTrade,127.886,127.796162,128.172394,25,2018-06-22 06:00:26,128.276757,...,128.276757,StopLossHit,BG_EntryBarState-Transistion,BG_EntryPriorBarState-BreakOut,FT_EntryBarState-BreakOut,FT_EntryPriorBarState-Congestion,BG_ExitBarStateCongestion,BG_ExitPriorBarStateCongestion,FT_ExitBarStateTransistion,FT_ExitPriorBarStateBreakOut


In [69]:
df = pd.read_csv(data_path, parse_dates=['TickDateTime']).set_index('TickDateTime')
df = df.loc[~df.index.duplicated(keep='first')]

In [70]:
def get_min_max_mae_of_bar(entry_time, exit_time, df):
    return df.loc[entry_time:exit_time,'HighPrice'].max(), df.loc[entry_time:exit_time,'LowPrice'].min()

In [71]:
for row_index, row in trade_df.iterrows():
    trade_entry_time = row['EntryDateTime']
    trade_exit_time = row['ExitDateTime']

    try:
        df.loc[trade_entry_time]
    except KeyError:
        print('Unable to find entry time, use the next available')
        trade_entry_time = df.loc[trade_entry_time:].index[0]
#     print(f'{trade_entry_time}, {row_index}')

    try:
        df.loc[trade_exit_time]
    except KeyError:
        print('Unable to find exit time, use the next available')
        trade_exit_time = df.loc[trade_exit_time:].index[0]

    bg_bar_number_name = 'BgHtfBarNumber'
    bg_entry_bar_number = df.loc[trade_entry_time, bg_bar_number_name]
    bg_exit_bar_number = df.loc[trade_exit_time, bg_bar_number_name]
    
    max_value, min_value = get_min_max_mae_of_bar(trade_entry_time, trade_exit_time, df)
    
    if row['Direction'] == 'Long':
        trade_df.loc[row_index, 'MAE'] = min_value
        trade_df.loc[row_index, 'MFE'] = max_value
    else:
        trade_df.loc[row_index, 'MAE'] = max_value
        trade_df.loc[row_index, 'MFE'] = min_value
    
    try:
        for i in range(0,5):
            _entry_index = df[df[bg_bar_number_name] == bg_entry_bar_number + i].index[-1] 
            max_value, min_value = get_min_max_mae_of_bar(trade_entry_time, _entry_index, df)
            trade_df.loc[row_index, f'EntryBar{i+1}Price'] = df.loc[_entry_index, 'ClosePrice']
            
            if row['Direction'] == 'Long':
                trade_df.loc[row_index, f'P{i+1}_EntryMAE'] = min_value
                trade_df.loc[row_index, f'P{i+1}_EntryMFE'] = max_value
            else:
                trade_df.loc[row_index, f'P{i+1}_EntryMAE'] = max_value
                trade_df.loc[row_index, f'P{i+1}_EntryMFE'] = min_value
        
        for i in range(0,5):
            _exit_index = df[df[bg_bar_number_name] == bg_exit_bar_number + i].index[-1] 
            max_value, min_value = get_min_max_mae_of_bar(trade_exit_time, _exit_index, df)
            trade_df.loc[row_index, f'ExitBar{i+1}Price'] = df.loc[_exit_index, 'ClosePrice']
            
            if row['Direction'] == 'Long':
                trade_df.loc[row_index, f'P{i+1}_ExitMAE'] = min_value
                trade_df.loc[row_index, f'P{i+1}_ExitMFE'] = max_value
            else:
                trade_df.loc[row_index, f'P{i+1}_ExitMAE'] = max_value
                trade_df.loc[row_index, f'P{i+1}_ExitMFE'] = min_value
    except IndexError:
        pass

    fetch_columns = ['BgAtr', 'BgVWAP', 'BgVWAPMigrating', 'BgYellow', 'BgGreen', 'BgGreenPrior','BgVPOC', 'BgVPOCPrior', 'BgTrendState', 'BgTwoPeriodHigh', 'BgTwoPeriodLow', 'BgValueArea', 'FtAtr', 'FtVWAP', 'FtVWAPMigrating', 'FtYellow', 'FtGreen', 'FtGreenPrior', 'FtVPOC', 'FtVPOCPrior', 'FtTrendState', 'FtTwoPeriodHigh', 'FtTwoPeriodLow', 'FtValueArea', 'BgChannelTop', 'BgChannelBottom', 'FtChannelTop', 'FtChannelBottom', 'BgTrendRunnerNumber', 'FtTrendRunnerNumber', 'BgHtfBarCount', 'BgHtfBarCountPrior', 'FtHtfBarCount', 'FtHtfBarCountPrior', 'Htf2TrendRunnerNumber', 'Htf2VWAPMigrating', 'Htf2HtfBarCount', 'Htf2TrendState']
    for _column in fetch_columns:
        trade_df.loc[row_index, _column] = df.loc[trade_entry_time, _column]
    
    trade_df.loc[row_index, 'CarryingBgBarCount'] = df.loc[trade_exit_time, 'BgHtfBarNumber'] - df.loc[trade_entry_time, 'BgHtfBarNumber'] 


In [72]:
fixed_columns = ['Ticker','EntryDateTime','Direction','EntryReason','EntryPrice','InitialStopLoss','InitialProfitTarget','TimeFrame','ExitDateTime','PreventStopLoss','ProfitTargetCondition','ExitPrice','ExitReason','MAE','MFE','EntryBar1Price','P1_EntryMAE','P1_EntryMFE','EntryBar2Price','P2_EntryMAE','P2_EntryMFE','EntryBar3Price','P3_EntryMAE','P3_EntryMFE','EntryBar4Price','P4_EntryMAE','P4_EntryMFE','EntryBar5Price','P5_EntryMAE','P5_EntryMFE','ExitBar5Price','P5_ExitMAE','P5_ExitMFE','BgAtr','BgVWAP','BgVWAPMigrating','BgYellow','BgGreen','BgGreenPrior','BgVPOC','BgVPOCPrior','BgTrendState','BgTwoPeriodHigh','BgTwoPeriodLow','BgValueArea','FtAtr','FtVWAP','FtVWAPMigrating','FtYellow','FtGreen','FtGreenPrior','FtVPOC','FtVPOCPrior','FtTrendState','FtTwoPeriodHigh','FtTwoPeriodLow','FtValueArea','BgChannelTop','BgChannelBottom','FtChannelTop','FtChannelBottom','ExitBar1Price','P1_ExitMAE','P1_ExitMFE','ExitBar2Price','P2_ExitMAE','P2_ExitMFE','ExitBar3Price','P3_ExitMAE','P3_ExitMFE','ExitBar4Price','P4_ExitMAE','P4_ExitMFE']
new_columns = [c for c in trade_df.columns if c not in fixed_columns]

In [73]:
trade_processed_path = trade_path.replace('.txt', '_processed.csv')
trade_df[fixed_columns+new_columns].to_csv(trade_processed_path, index=False)